In [1]:
!pip install qiskit qiskit_aer qiskit-ibm-runtime matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.8 MB/s eta 0:00:00


In [2]:
# Bernstein–Vazirani Algorithm using Qiskit 2.x
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s, b):
    """
    Implements oracle for f(x) = s · x ⊕ b.

    The function f(x) is encoded by applying a Z-gate (phase kickback)
    to the ancilla qubit if f(x)=1.

    * s · x is implemented by CNOT gates from x_i to ancilla for each '1' in s.
    * ⊕ b is implemented by applying an X gate to the ancilla if b=1.
    """
    # 1. Implement s · x
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

    # 2. Implement ⊕ b
    # The ancilla qubit has |-> state. Applying an X-gate to it flips the phase
    # of the final state if b=1, effectively adding the constant b to the result.
    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_circuit(s, b='0'):
    """
    Constructs the Bernstein-Vazirani circuit for the secret string s and constant b.
    """
    n = len(s)
    # n input qubits and 1 ancilla qubit
    qreg = QuantumRegister(n + 1, 'q')
    # n classical bits for measurement
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # 1. Initialize ancilla to |-⟩ state: apply X and H
    # The ancilla is the last qubit, index n
    qc.x(ancilla)
    qc.h(ancilla)

    # 2. Apply H to all input qubits
    qc.h(inputs)

    # Add a barrier for visualization clarity
    qc.barrier()

    # 3. Apply the oracle
    bv_oracle(qc, inputs, ancilla, s, b)

    # Add a barrier
    qc.barrier()

    # 4. Apply H to all input qubits again
    qc.h(inputs)

    # 5. Measure the input qubits
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, s, b, shots=1024):
    """
    Runs the BV circuit on a Qiskit Aer simulator, prints results, and verifies.
    """
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print('--- Simulation Results ---')
    print(f'Secret string s = {s}, Constant b = {b}')
    print('Counts:', counts)

    # Display histogram
    fig = plot_histogram(counts, title="Bernstein-Vazirani Result")
    plt.show()

    # The result is the most frequent measured bitstring
    most_frequent_measured = max(counts, key=counts.get)
    print(f'Most frequent measured bitstring (input register): {most_frequent_measured}')

    if most_frequent_measured == s:
        print('✅ Successfully recovered secret string s')
        # The constant b affects only the phase of the final state, which is not
        # visible in the measurement of the input register.
        print(f'   The constant b="{b}" does not change the measurement of s.')
    else:
        print('⚠️ Measured string differs from s. Check circuit or noise.')

    return most_frequent_measured

# --- Main Execution ---
if __name__ == '__main__':

    # --- Task 1: Change the secret string s and verify ---
    s = '01101'
    b = '0' # Constant b = 0
    print('*** Running with Secret String s = 01101 and b = 0 ***')
    qc = bernstein_vazirani_circuit(s, b)
    print(qc.draw(output='text', fold=-1, idle_wires=False))
    measured = run_bv(qc, s, b)

    # --- Task 2: Modify the oracle to include a constant bit b (f(x) = s·x ⊕ b) ---
    # The oracle modification is already implemented in bv_oracle and
    # bernstein_vazirani_circuit by including the 'b' parameter and the X-gate
    # conditional on b='1'.

    s = '101'
    b = '1' # Constant b = 1
    print('\n*** Running with Secret String s = 101 and constant b = 1 (f(x) = s·x ⊕ 1) ***')
    qc_b1 = bernstein_vazirani_circuit(s, b)
    print(qc_b1.draw(output='text', fold=-1, idle_wires=False))
    measured_b1 = run_bv(qc_b1, s, b)

    # The effect of b=1 is that the ancilla qubit (q_3 in this 4-qubit circuit)
    # receives an initial X-gate *before* the Hadamard gates, making its initial
    # state |0>.
    # WAIT - this is *incorrect* for the standard implementation of BV with b.
    # The standard implementation for f(x) = s·x ⊕ b requires the ancilla
    # to be initialized to |-> and then the operation U_f|x>|y> = |x>|y ⊕ f(x)>
    # is implemented.
    # The X gate for constant b='1' is typically placed *after* the s·x CNOTs
    # in the middle of the circuit (in U_f).

    # Let's verify the updated bv_oracle logic:
    # Ancilla state before U_f is |-> = (|0> - |1>)/sqrt(2)
    # U_f = Z on ancilla if f(x)=1.
    # If b=1, we add an X gate on the ancilla after s·x CNOTs:
    # U_f |x>|-> = U_{s·x ⊕ b} |x>|->
    # U_{s·x ⊕ b} is composed of U_{s·x} (CNOTs) followed by X (if b=1).
    # X|-> = X (|0> - |1>)/sqrt(2) = (|1> - |0>)/sqrt(2) = -|->
    # This means X on the ancilla applies a global phase of -1 to the final state,
    # which is not measured.
    # *However, in the given implementation, the X for b='1' is correctly applied
    # only if the oracle function should output 1 for input 0 (which is b).*
    # The provided code places the X *within* bv_oracle, which is the correct
    # place to implement the constant b.


    # --- Task 3 & 4: Real Backend and Noise (Code Only) ---
    print('\n*** Code for Real Backend and Noise (Not Executed) ***')

    # --- Task 3: Run on a real IBM backend (qiskit_ibm_runtime) ---
    # This requires an IBM Quantum Platform account and a saved API token.
    print('\n--- Real Backend Code (Requires Authentication) ---')
    print("> from qiskit_ibm_runtime import QiskitRuntimeService, Sampler")
    print("> # service = QiskitRuntimeService(channel=\"ibm_quantum\")")
    print("> # backend = service.get_backend(\"ibm_brisbane\") # Example backend")
    print("> # sampler = Sampler(backend=backend)")
    print("> # job = sampler.run(qc, shots=1024)")
    print("> # print(job.result())")

    # --- Task 4: Add noise via qiskit_aer.noise.NoiseModel ---
    print('\n--- Noise Model Code ---')
    print("> from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error")
    print("> from qiskit_aer import AerSimulator")
    print("> ")
    print("> # Define a simple depolarizing noise model")
    print("> noise_model = NoiseModel()")
    print("> p_depol = 0.01  # Depolarizing probability")
    print("> error_1q = depolarizing_error(p_depol, 1)")
    print("> error_2q = depolarizing_error(p_depol, 2)")
    print("> ")
    print("> # Add errors to all single and two-qubit gates")
    print("> noise_model.add_all_qubit_quantum_error(error_1q, ['u', 'rz', 'sx', 'x'])")
    print("> noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])")
    print("> ")
    print("> # Run with noise")
    print(f"> sim_noise = AerSimulator(noise_model=noise_model)")
    print(f"> tqc_noise = transpile(qc, sim_noise)")
    print(f"> job_noise = sim_noise.run(tqc_noise, shots=1024)")
    print(f"> counts_noise = job_noise.result().get_counts()")
    print(f"> print('Noisy Counts:', counts_noise)")
    print(f"> # plot_histogram(counts_noise)")

*** Running with Secret String s = 01101 and b = 0 ***
     ┌───┐      ░                 ░ ┌───┐┌─┐            
q_0: ┤ H ├──────░─────────────────░─┤ H ├┤M├────────────
     ├───┤      ░                 ░ ├───┤└╥┘┌─┐         
q_1: ┤ H ├──────░───■─────────────░─┤ H ├─╫─┤M├─────────
     ├───┤      ░   │             ░ ├───┤ ║ └╥┘┌─┐      
q_2: ┤ H ├──────░───┼────■────────░─┤ H ├─╫──╫─┤M├──────
     ├───┤      ░   │    │        ░ ├───┤ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├──────░───┼────┼────────░─┤ H ├─╫──╫──╫─┤M├───
     ├───┤      ░   │    │        ░ ├───┤ ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├──────░───┼────┼────■───░─┤ H ├─╫──╫──╫──╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌─┴─┐┌─┴─┐ ░ └───┘ ║  ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├─░─┤ X ├┤ X ├┤ X ├─░───────╫──╫──╫──╫──╫─
     └───┘└───┘ ░ └───┘└───┘└───┘ ░       ║  ║  ║  ║  ║ 
c: 5/═════════════════════════════════════╩══╩══╩══╩══╩═
                                          0  1  2  3  4 
--- Simulation Results ---
Secret string s = 01101, Constant b = 0
Counts: {'10110': 1024}